In [18]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x)) # same as reshape in numpy
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]    # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        
        return num_features

nnet = Net()
print(nnet)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


Backward function (where gradients are computed) is automatically defined using autograd.

In [11]:
params = list(nnet.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


The input to the forward is an autograd.Variable, and so is the output.

In [12]:
input = Variable(torch.randn(1, 1, 32, 32))
out = nnet(input)
print(out)

Variable containing:
-0.0150 -0.0312  0.0880 -0.1075 -0.0490 -0.1126  0.0767  0.0266 -0.0528 -0.0323
[torch.FloatTensor of size 1x10]



In [13]:
nnet.zero_grad()
out.backward(torch.randn(1, 10))

## Loss Function

In [14]:
output = nnet(input)
target = Variable(torch.arange(1, 11)) # a dummy target
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.7474
[torch.FloatTensor of size 1]



In [15]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [17]:
nnet.zero_grad() # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(nnet.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(nnet.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
  2.0011
 -4.2485
 -4.6655
  2.1558
  5.4188
 -5.7582
[torch.FloatTensor of size 6]



The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

    weight = weight - learning_rate * gradient

In [21]:
learning_rate = 0.01
for f in nnet.parameters():
    f.data.sub_(f.grad.data*learning_rate)

In [22]:
optimizer = optim.SGD(nnet.parameters(), lr=0.01)

optimizer.zero_grad()
output = nnet(input)
loss = criterion(output, target)
loss.backward
optimizer.step()